# LN - Data PP - Stability and efficiency calculations

## Import libraries and data

In [30]:
import sqlite3
import numpy as np
import pandas as pd
import networkx as nx
import itertools
#import matplotlib.pyplot as plt
import time
import pickle

import os
import re
import sys

from tqdm.notebook import trange, tqdm
from time import sleep

from dask_cloudprovider import FargateCluster
from dask.distributed import Client
import dask.array as da
import dask

import s3fs

import boto3


In [16]:
directory='/Users/dsrincon/Dropbox/Personal/EDU/Posgrado/masters_usa/MIMS/research/MIMS_capstone'

In [17]:


# Load Data

# Dataframe
#decisions_df=pd.read_csv(directory+'/data/LN_channels.csv')

# Channel closures
#with open(directory+'/data/channel_closures.p', 'rb') as pickle_file:
 #   channel_closures = pickle.load(pickle_file)
    
    
# Channel openings     
#with open(directory+'/data/channel_opens.p', 'rb') as pickle_file:
 #   channel_opens = pickle.load(pickle_file)
    

# Create list with graph keys

# graph_dir='./data/graph_snapshots' - For local tests
extraction_id=1585344554

# Create s3 resource to access data
s3 = boto3.resource('s3')
graph_keys = [obj.key 
        for obj in s3.Bucket(name='ln-strategy-data').objects.all()
        if re.match(".*"+str(extraction_id)+"_connected/.*\.gpickle",obj.key)]



In [6]:
# Test Data Load
print(len(graph_keys))

3988


In [4]:
decisions_df.head()

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,close_time,close_fee,last_update,close_type,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id
0,505149x622x0,505149,f6bc767df9148ebf76d2b9baf4eb46e3230712c2bf5a51...,bc1qjmg6ev344fenh3zhg0yjl6hyvxpxluw6x9nn2a5lv4...,592638.0,82cb2ea2a06c8c453d8b9ca08e17bbefe87225aa380b2d...,0250373555232cec757ea141273e75381c84cc3ab22f1e...,02ef61a252f9504a42fc264a28476f44cea0711a44b2da...,300000,2019-08-22 02:49:00,...,1.567276e+09,184.0,1.563172e+09,mutual,0.0,3570.0,296246.0,0,3098,1492
1,513675x2245x0,513675,4297b5fe9beeb701c67fd0f84861b22edbcafe5c25be67...,bc1qymmdt0vzhdjqyqw2cevrqppp6rrlg5j2l20yk72z6y...,594718.0,3f86d9427c750f37a963b5a329da8941520f5a6cdbfe02...,028aa5a991a2acf33da91674fe062219b640e5e57d77a4...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,50000,2019-10-07 02:42:58,...,1.568401e+09,4410.0,1.552879e+09,unused,0.0,45590.0,0.0,1,5474,7365
2,513887x1177x0,513887,3b4cc434e62c1739e79171c7c1641bf9ac0e32d8530c68...,bc1q48l3h7sfdjaqat3sy98naltkwlujwefwnkfqxfm8fd...,NaN,NaN,02d97e94cfeedca2a3da47acb400bc6836e671b3cb3fc0...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,50000,2020-02-14 03:15:22,...,NaN,NaN,1.581536e+09,NaN,NaN,NaN,NaN,2,2797,7365
3,513909x1248x0,513909,86311514680351b1e644276efd7704ba13be169cc1a272...,bc1q24kvd9wdjdhwgr54fmu7cu9xldmsjuwdsq2ph5fwnj...,NaN,NaN,02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,20000,2020-02-14 03:15:26,...,NaN,NaN,1.581464e+09,NaN,NaN,NaN,NaN,3,211,7365
4,513910x1814x1,513910,7f010765ce336d2be78c846844544e6a06ce2c59e7785f...,bc1qzh9xrpqvyse7fuanc8tl5e75qymq5fzk3deh78hs02...,NaN,NaN,02ad6fb8d693dc1e4569bcedefadf5f72a931ae027dc0f...,03fab7f8655169ea77d9691d4bd359e97782cb6177a6f7...,20000,2020-02-14 03:15:26,...,NaN,NaN,1.581466e+09,NaN,NaN,NaN,NaN,4,211,7365


### Connection to AWS S3

## Setup Fargate Cluster

In [61]:
cluster = FargateCluster(n_workers=70,scheduler_timeout='30 minutes',image='dsrincon/dask-graph:nx-scipy-v1')

Task exception was never retrieved
future: <Task finished coro=<_wrap_awaitable() done, defined at /opt/conda/lib/python3.7/asyncio/tasks.py:596> exception=ClientError('An error occurred (ThrottlingException) when calling the DescribeTasks operation (reached max retries: 4): Rate exceeded')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/asyncio/tasks.py", line 603, in _wrap_awaitable
    return (yield from awaitable.__await__())
  File "/opt/conda/lib/python3.7/site-packages/dask_cloudprovider/providers/aws/ecs.py", line 130, in _
    await self.start()
  File "/opt/conda/lib/python3.7/site-packages/dask_cloudprovider/providers/aws/ecs.py", line 240, in start
    await self._update_task()
  File "/opt/conda/lib/python3.7/site-packages/dask_cloudprovider/providers/aws/ecs.py", line 158, in _update_task
    cluster=self.cluster_arn, tasks=[self.task_arn]
  File "/opt/conda/lib/python3.7/site-packages/aiobotocore/client.py", line 90, in _make_api_call
    raise error

In [62]:
cluster

In [63]:
client = Client(cluster)

/opt/conda/lib/python3.7/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

python
+---------------------------+---------------+
|                           | version       |
+---------------------------+---------------+
| client                    | 3.7.3.final.0 |
| scheduler                 | 3.7.4.final.0 |
| tcp://172.31.1.5:39671    | 3.7.4.final.0 |
| tcp://172.31.12.222:46663 | 3.7.4.final.0 |
| tcp://172.31.12.74:37731  | 3.7.4.final.0 |
| tcp://172.31.18.154:38807 | 3.7.4.final.0 |
| tcp://172.31.18.163:45357 | 3.7.4.final.0 |
| tcp://172.31.18.221:42953 | 3.7.4.final.0 |
| tcp://172.31.19.45:36321  | 3.7.4.final.0 |
| tcp://172.31.2.26:33719   | 3.7.4.final.0 |
| tcp://172.31.20.182:33445 | 3.7.4.final.0 |
| tcp://172.31.23.149:45755 | 3.7.4.final.0 |
| tcp://172.31.26.156:42045 | 3.7.4.final.0 |
| tcp://172.31.26.157:33863 | 3.7.4.final.0 |
| tcp://172.31.27.68:32805  | 3.7.4.final.0 |
| tcp://172.31.29.11:44009  | 3.7.4.final.0 |
|

In [69]:
# Lazy extract Graphs

# Function for lazy S3 extraction
def load_snapshots(key):
    session = boto3.session.Session()
    s3 = session.resource('s3')
    response = s3.Object(bucket_name='ln-strategy-data', key=key).get()
    G=pickle.loads(response['Body'].read())
    
    return G
    
# Script to create delayed array
graph_snapshots=[]

for key in graph_keys[700:2000]: # Remove index for full range
    G=dask.delayed(load_snapshots)(key)
    graph_snapshots.append(G)
   

In [ ]:
# Test Lazy Graph extract
# graph_snapshots=dask.compute(*graph_snapshots) - Uncomment 
#print(len(graph_snapshots[5]))
#print(graph_snapshots[3].graph['block'])

# Delayed testing
#results = dask.compute(*futures)
#graphs=dask.compute(*graph_snapshots)

## Stability/Efficiency analysis by utility definition

In order to understand the potential motivations behind each decision we analyze each decission (opening or closure of a channel) independently from the perspective of each of the participants in the decission, which we'll call the node under analysis. For each decission we extract or compute the following information: 



## Betweeness


Betweenness centrality measures how central is a network to the flow of information in a network. In the case of the Lightning Network the higher the betweenness centrality of a node, the more transactions (messages) that are routed through it. In particular, we will use a measure of betweenness centrality defined in (Brandes and Fleischer 2005 - https://link.springer.com/chapter/10.1007/978-3-540-31856-9_44) that models infomation through a network, as electric current, efficiently and not only considering shortest path. This allows us to account for the fact that not all transactions travel through shortes path given that there are fee and capacity considerations.  

### Baseline betweeness

In [70]:
# Test networkx (sequential computation)

snapshot_bet=[]

def bet_cent(g):
    #if len(g)>2:
    g_bet=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(g,weight='capacity',kmax=10000)
    #else:
    #g_bet={}
    return g_bet
    

for g in graph_snapshots:
    g_bet=dask.delayed(bet_cent)(g)
    snapshot_bet.append(g_bet)

futures = dask.persist(*snapshot_bet)

In [71]:
start=time.time()
results = dask.compute(*futures)
end=time.time()
print('Compute in seconds: {}'.format(end-start))

Compute in seconds: 39.340877056121826


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [36]:
# Test results
# Create list with graph keys
print(sys.getsizeof(results))

31952


### Pairwise stability 

- **Marginal % change in betweenness (bet_mar_deltai)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting the decission (adding or removing a channel). Weighted current betweenness centrality is used for this measure.

In [ ]:
# For each block get snapshot

    # For each snapshot get all OPENS. 

    # For each open get betweenness for each node 

    # For each open 





- **Actual % change in betweenness (bet_act_deltai)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted current betweenness centrality is used for this measure.

- **Marginal betweeness pairwise stability (bet_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a betweenness perspective.

- **Actual betweeness pairwise stability (bet_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a betweenness perspective. 




### Nash stability 

- **% Change with respect to not making decision (bet_binstat_deltai)**: The % change in betwewnness centrality, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (bet_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher betweenness centrality. This tells me if my strategy helped me be better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (bet_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher betwneenness centrality. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (bet_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower betwneenness centrality. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average betweenness centrality for all the nodes. 



## Connectivity

### Pairwise stability 

- **Marginal % change in connectivity (con_mar_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting the decission (adding or removing a channel). Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Actual % change in connectivity (con_act_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Marginal connectivity pairwise stability (con_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a connectivity perspective.

- **Actual connectivity pairwise stability (con_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a connectivity perspective.  



### Nash stability 

- **% Change with respect to not making decision (con_binstat_deltai)**: The % change in shortest path average, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (con_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher shortest path average. NOTE: This indicates if the strategy selected made the node better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (con_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher shortest path average. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (con_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower shortest path average. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average shortest path average for all the nodes. 



